In [12]:
import pandas as pd

from warnings import filterwarnings
filterwarnings("ignore")

In [13]:
# import data
users = pd.read_csv("data_raw/USERS.csv")
orders = pd.read_csv("data_raw/ORDERS.csv")
items  = pd.read_csv("data_raw/ORDERITEMS.csv")
promos = pd.read_csv("data_raw/ORDER_PROMOTIONS.csv")
products = pd.read_csv("data_raw/PRODUCT_DETAILS.csv")

In [14]:
# robust datetime parsing (handles formats like 2021-10-22-15.20.33.000000)
DT_FMT = "%Y-%m-%d-%H.%M.%S.%f"

# Users table dates
for col in ["REGISTRATION","LASTSESSION","REGISTRATIONUPDATE","LASTORDER"]:
    if col in users.columns:
        users[col] = pd.to_datetime(users[col], format=DT_FMT, errors="coerce").dt.date

# Orders table dates (THIS is what drives recency later)
for col in ["TIMEPLACED","LASTUPDATE"]:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], format=DT_FMT, errors="coerce").dt.date

# Promotions table dates
for col in ["STARTDATE","ENDDATE"]:
    if col in promos.columns:
        promos[col] = pd.to_datetime(promos[col], format=DT_FMT, errors="coerce").dt.date

In [15]:
# deduplicate data
users = users.sort_values(["USERS_ID","REGISTRATIONUPDATE","LASTSESSION"]).drop_duplicates("USERS_ID", keep="last")

products = products.drop_duplicates()  # exact full-row duplicates
products = products.drop_duplicates("PARTNUM", keep="first")

orders = orders.drop_duplicates()
orders = orders.sort_values(["ORDERS_ID","LASTUPDATE"]).drop_duplicates("ORDERS_ID", keep="last")

items = items.drop_duplicates()
items = items.drop_duplicates("ORDERITEMS_ID", keep="first")

promos = promos.drop_duplicates()

In [16]:
# combine first and last name and drop individual columns
users["NAME"] = users["FIRSTNAME"].str.strip() + " " + users["LASTNAME"].str.strip()
users = users.drop(columns=["FIRSTNAME","LASTNAME"])

In [17]:
# create the fact tables
fact_orders = orders.copy()
fact_orderitems = items.copy()

# promo rollup at item level (keeps multi-promos but also gives item summary)
promo_item = (
    promos.groupby("ORDERITEMS_ID")
    .agg(
        promo_count=("PROMOTION_NAME","nunique"),
        promo_types=("TYPE","nunique"),
        promo_adj_total=("TOTALADJUSTMENT","sum")
    )
    .reset_index()
)

In [18]:
# create trasaction table
fact_transactions = (
    fact_orderitems
    .merge(fact_orders, on="ORDERS_ID", how="left", suffixes=("_item","_order"))
    .merge(products, on="PARTNUM", how="left")
    .merge(promo_item, on="ORDERITEMS_ID", how="left")
)

# fill promo nulls
fact_transactions[["promo_count","promo_types","promo_adj_total"]] = (
    fact_transactions[["promo_count","promo_types","promo_adj_total"]].fillna(0)
)

In [19]:
users.to_csv("data_intermediate/dim_users.csv", index=False)
products.to_csv("data_intermediate/dim_products.csv", index=False)
fact_orders.to_csv("data_intermediate/fact_orders.csv", index=False)
fact_orderitems.to_csv("data_intermediate/fact_orderitems.csv", index=False)
fact_transactions.to_csv("data_intermediate/fact_transactions.csv", index=False)